Projeto Final:\
\
Guia para Legend of Zelda: Breath of the Wild

In [1]:
from plyer import notification
from datetime import datetime
import pandas as pd
import requests
import sqlite3

In [2]:
def alerta(nivel, base, etapa):
    #função de alerta caso tenha algum erro durante o processamento

    hoje = datetime.now()
    hoje = hoje.strftime("%d/%m/%Y %H:%M:%S")

    message = f'Falha no carregamento da base {base} na etapa {etapa}.\n{hoje}'
    
    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    
    notification.notify(
        title = title,
        message = message,
        app_name = 'alerta',
        timeout = 10)

Banco de Dados

In [3]:
def retrieve_table():
    #confere as tabelas criadas
    conn = sqlite3.connect('zelda.db')
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()
    return schema

def save_db(df,table_name):
    #armazena as tabelas no banco de dados
    conn = sqlite3.connect('zelda.db')
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    conn.close()
    return True

def load_db(query):
    #acessa as tabelas armazenadas no banco de dados
    conn = sqlite3.connect('zelda.db')
    df = pd.read_sql(query, conn)

    conn.close()
    return df

Extração

In [4]:
def get_api(url):
    #função de acesso a API
    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        data_json = data_json['data'] #pequena correção dada a particularidade da api usada
        return data_json
    else:
        print(f'Erro {response.status_code}')

In [5]:
def get_item():
    #criação da tabela item
    url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/all'
    data_json = get_api(url)
    compendium_df = pd.DataFrame(data_json)
    id          = [id for id in compendium_df['id']]
    name        = [name for name in compendium_df['name']]
    locations   = [loc for loc in compendium_df['common_locations']]
    drops       = [drop for drop in compendium_df['drops']]

    item_df = pd.DataFrame({
        'id': id,
        'name': name,
        'locations': locations,
        'drops': drops})
    item_df = item_df.explode('locations', ignore_index=True).explode('drops', ignore_index=True).dropna().sort_values(by='id', ascending=True)
    #extrai os dados de uma lista, ignora dados nulos e ordena pelo id
    return item_df

def get_food():
    #criação da tabela food
    url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/all'
    data_json = get_api(url)
    compendium_df = pd.DataFrame(data_json)
    id          = [id for id in compendium_df['id']]
    name        = [name for name in compendium_df['name']]
    locations   = [loc for loc in compendium_df['common_locations']]
    effect      = [fx for fx in compendium_df['cooking_effect']]
    heal        = [heal for heal in compendium_df['hearts_recovered']]

    food_df = pd.DataFrame({
        'id': id,
        'name': name,
        'locations': locations,
        'effect': effect,
        'heal': heal})
    food_df = food_df.explode('locations', ignore_index=True).dropna().sort_values(by='id', ascending=True)
    #extrai os dados de uma lista, ignora dados nulos e ordena pela id
    return food_df

def get_equipment():
    #criação da tabela equipment
    url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/category/equipment'
    data_json = get_api(url)
    df_equipment = pd.DataFrame(data_json)
    properties = list(df_equipment['properties']) #lista uma coluna em formato dict para separar os dados para nova tabela
    df_properties = pd.DataFrame(properties)
    df_equipment = pd.concat([df_equipment, df_properties], axis=1) #concatena a tabela nova com a anterior
    id          = [id for id in df_equipment['id']]
    name        = [name for name in df_equipment['name']]
    locations   = [loc for loc in df_equipment['common_locations']]
    attack      = [atk for atk in df_equipment['attack']]
    defense     = [dfc for dfc in df_equipment['defense']]

    equipment_df = pd.DataFrame({
        'id': id,
        'name': name,
        'locations': locations,
        'attack': attack,
        'defense': defense})
    equipment_df = equipment_df.explode('locations', ignore_index=True).fillna(0).sort_values(by='id', ascending=True)
    #extrai os dados de uma lista, preenche nulos com zeros e ordena pela id
    return equipment_df

def get_compendium():
    #criação da tabela bruta
    url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/all'
    data_json = get_api(url)
    compendium_df = pd.DataFrame(data_json)
    
    return compendium_df

In [6]:
def extração():
    #função que executa todas as etapas de extração e criação de tabelas para armazenar no banco de dados
    try:
        item_df = get_item()
        save_db(item_df, 'item')
    except:
        alerta(
            nivel = 3,
            base = 'item',
            etapa = 'extração')
    try:
        food_df = get_food()
        save_db(food_df, 'food')
    except:
        alerta(
            nivel = 3,
            base = 'food',
            etapa = 'extração')
    try:
        equipment_df = get_equipment()
        save_db(equipment_df, 'equipment')
    except:
        alerta(
            nivel = 3,
            base = 'equipment',
            etapa = 'extração')

Tratamento

In [7]:
def tratamento():
    #função de tratamento e limpeza da tabela bruta
    try:
        compendium_df = get_compendium()
        columns_drop = ['dlc', 'image', 'edible']
        compendium_df = compendium_df.drop(columns=columns_drop, axis=1).fillna('')
        #dispensa tabelas com dados irrelevantes e preenche o restante nulo com espaço vazio
        columns_astype = ['common_locations', 'drops', 'properties']
        compendium_df[columns_astype] = compendium_df[columns_astype].apply(lambda i: i.astype('str'))
        #converte colunas em formato incompatível com sql
        save_db(compendium_df, 'compendium')
    except:
        alerta(
            nivel = 2,
            base = 'compendium',
            etapa = 'tratamento')

Executar

In [8]:
extração()

In [9]:
tratamento()

In [10]:
retrieve_table()

,name
0,item
1,food
2,equipment
3,compendium


Consulta

In [17]:
df = load_db(query="SELECT * FROM equipment WHERE locations = 'Hyrule Field' AND attack > 0 ORDER BY attack DESC")
df.head()

,id,name,locations,attack,defense
0,235,savage lynel sword,Hyrule Field,58.0,0.0
1,264,edge of duality,Hyrule Field,50.0,0.0
2,278,ancient battle axe+,Hyrule Field,45.0,0.0
3,260,stone smasher,Hyrule Field,42.0,0.0
4,234,mighty lynel sword,Hyrule Field,36.0,0.0


In [18]:
load_db(query="SELECT * FROM food WHERE name = 'chillshroom'")

,id,name,locations,effect,heal
0,179,chillshroom,Mount Lanayru,heat resistance,0.5
1,179,chillshroom,Hebra Mountains,heat resistance,0.5


In [19]:
load_db(query="SELECT * FROM equipment").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         342 non-null    int64  
 1   name       342 non-null    object 
 2   locations  342 non-null    object 
 3   attack     342 non-null    float64
 4   defense    342 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 13.5+ KB


In [20]:
load_db(query="SELECT * FROM compendium")

,category,common_locations,description,drops,id,name,cooking_effect,hearts_recovered,properties
0,treasure,['Greater Hyrule'],This deposit contains quite a bit of luminous ...,"['luminous', 'flint']",389,luminous ore deposit,,,
1,treasure,['Greater Hyrule'],This deposit contains a good deal of precious ...,"['ruby', 'sapphire', 'diamond', 'amber', 'topa...",388,rare ore deposit,,,
2,treasure,['Greater Hyrule'],This deposit contains a good deal of ore. Brea...,"['ruby', 'diamond', 'amber', 'sapphire', 'topa...",387,ore deposit,,,
3,treasure,['Greater Hyrule'],Fortunes untold (potentially) await the lucky ...,['treasures'],386,treasure chest,,,
4,monsters,['Divine Beast Vah Naboris'],This phantom of Ganon attacked the Divine Beas...,[],159,thunderblight ganon,,,
...,...,...,...,...,...,...,...,...,...
384,creatures,"['Hebra Mountains', 'Tabantha Frontier']",This particular breed of grassland fox makes i...,"['raw prime meat', 'raw gourmet meat']",20,snowcoat fox,,,
385,creatures,['Gerudo Desert'],This is Riju's own sand seal. It may look inte...,[],8,patricia,,,
386,creatures,"['Hebra Mountains', 'Tabantha Frontier', 'Rito...",These small birds live in the Hebra region and...,['chickaloo tree nut'],31,red sparrow,,,
387,creatures,"['Hyrule Field', 'Hyrule Ridge']","As the name suggests, these birds are rather c...",['chickaloo tree nut'],30,common sparrow,,,
